# Method Resolution Order

Inheritance is a tool for code reuse.

Python super() is different than other languages. Python's super() doesn't just call the parent of the class. Python MRO **linearize** the ancestors.

In [2]:
class Adam(object): pass
class Eve(object): pass
class Ramon(Adam, Eve): pass
class Gayle(Adam, Eve): pass
class Raymond(Ramon, Gayle): pass
class Dennis(Adam, Eve): pass
class Sharon(Adam, Eve): pass
class Rachel(Dennis, Sharon): pass
class Matthew(Raymond, Rachel): pass

Show MRO (Method resolution order) for a multi-level diamond diagram

In [4]:
help(Matthew)

Help on class Matthew in module __main__:

class Matthew(Raymond, Rachel)
 |  Method resolution order:
 |      Matthew
 |      Raymond
 |      Ramon
 |      Gayle
 |      Rachel
 |      Dennis
 |      Sharon
 |      Adam
 |      Eve
 |      builtins.object
 |  
 |  Data descriptors inherited from Adam:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



# Example 1

In [12]:
class DoughFactory(object):
    
    def get_dough(self):
        return 'insecticide treated wheat dough'
    
class Pizza(DoughFactory):
    
    def order_pizza(self, *toppings):
        print('Getting dough')
        dough = super().get_dough()
        print('Making pie with %s' % dough)
        for topping in toppings:
            print('Adding: %s' % topping)

In [14]:
Pizza().order_pizza('Pepperoni', 'Bell Pepper')

Getting dough
Making pie with insecticide treated wheat dough
Adding: Pepperoni
Adding: Bell Pepper


In [15]:
help(Pizza)

Help on class Pizza in module __main__:

class Pizza(DoughFactory)
 |  Method resolution order:
 |      Pizza
 |      DoughFactory
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  order_pizza(self, *toppings)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DoughFactory:
 |  
 |  get_dough(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from DoughFactory:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [16]:
class OrganicDoughFactory(DoughFactory):
    def get_dough(self):
        return 'pure untreated wheat dough'
    
class OrganicPizza(Pizza, OrganicDoughFactory):
    pass

In OrganicPizza, **super()** in class Pizza is decided by OrganicPizza (the child of Pizza) instead of Pizza.

The linearization algorithm guarantees that OrganicDoughFactory goes before DoughFactory

In [17]:
OrganicPizza().order_pizza('Saussage', 'Mushroom')

Getting dough
Making pie with pure untreated wheat dough
Adding: Saussage
Adding: Mushroom


In [18]:
help(OrganicPizza)

Help on class OrganicPizza in module __main__:

class OrganicPizza(Pizza, OrganicDoughFactory)
 |  Method resolution order:
 |      OrganicPizza
 |      Pizza
 |      OrganicDoughFactory
 |      DoughFactory
 |      builtins.object
 |  
 |  Methods inherited from Pizza:
 |  
 |  order_pizza(self, *toppings)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from OrganicDoughFactory:
 |  
 |  get_dough(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from DoughFactory:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



# Example 2: Dependency Injection

In [19]:
class Robot(object):
    def fetch(self, tool):
        print('Physical Movement! Fetching')
    def move_forward(self, tool):
        print('Physical Movement! Moving forward')
    def move_backward(self, tool):
        print('Physical Movement! Moving backward')
    def replace(self, tool):
        print('Physical Movement! Replacing')

In [64]:
class CleaningRobot(Robot):
    
    def clean(self, tool, times=3):
        super().fetch(tool)
        for i in range(times):
            super().move_forward(tool)
            super().move_backward(tool)
        super().replace(tool)

In [65]:
t = CleaningRobot()
t.clean('broom')

Physical Movement! Fetching
Physical Movement! Moving forward
Physical Movement! Moving backward
Physical Movement! Moving forward
Physical Movement! Moving backward
Physical Movement! Moving forward
Physical Movement! Moving backward
Physical Movement! Replacing


Mock Robot to test instead of use the real Robot.

Problem: CleaningRobot is hardwired with Robot because of the inheritance -> Cleaning Robot **is a** Robot, not **has a** robot

In [66]:
class MockBot(Robot):
    def __init__(self):
        self.tasks = []
    def fetch(self, tool):
        self.tasks.append('fetching %s' % tool)
    def move_forward(self, tool):
        self.tasks.append('forward %s' % tool)
    def move_backward(self, tool):
        self.tasks.append('backward %s' % tool)
    def replace(self, tool):
        self.tasks.append('replace %s' % tool)

In [67]:
class MockedCleaningRobot(CleaningRobot, MockBot):
    pass

In [68]:
t = MockedCleaningRobot()
t.clean('broom')

expected = ['fetching broom'] + ['forward broom', 'backward broom'] * 3 + ['replace broom']

t.tasks == expected

True

In [31]:
help(MockedCleaningRobot)

Help on class MockedCleaningRobot in module __main__:

class MockedCleaningRobot(CleaningRobot, MockBot)
 |  Method resolution order:
 |      MockedCleaningRobot
 |      CleaningRobot
 |      MockBot
 |      Robot
 |      builtins.object
 |  
 |  Methods inherited from CleaningRobot:
 |  
 |  clean(self, tool, times=3)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from MockBot:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  fetch(self, tool)
 |  
 |  move_backward(self, tool)
 |  
 |  move_forward(self, tool)
 |  
 |  replace(self, tool)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Robot:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



# Example 3

In [39]:
from collections import Counter, OrderedDict

In [42]:
class OrderedCounter(Counter, OrderedDict):
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, OrderedDict(self))
    
    def __reduce__(self):
        return self.__class__, (OrderedDict(self), )

In [43]:
oc = OrderedCounter('abcdadfaadsf')
print(oc)

OrderedCounter(OrderedDict([('a', 4), ('b', 1), ('c', 1), ('d', 3), ('f', 2), ('s', 1)]))


# Example 4

pass arguments with **kwds**

In [75]:
class Shape:
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)        

class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)

cs = ColoredShape(color='red', shapename='circle')

In [79]:
ColoredShape.__mro__

(__main__.ColoredShape, __main__.Shape, object)

Create a Root class with draw() so Shape can call super().draw()

In [80]:
class Root:
    def draw(self):
        # the delegation chain stops here
        assert not hasattr(super(), 'draw')

class Shape(Root):
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)
    def draw(self):
        print('Drawing.  Setting shape to:', self.shapename)
        super().draw()

class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)
    def draw(self):
        print('Drawing.  Setting color to:', self.color)
        super().draw()

cs = ColoredShape(color='blue', shapename='square')
cs.draw()

Drawing.  Setting color to: blue
Drawing.  Setting shape to: square


Incorporate a non-cooperative class

In [81]:
class Moveable:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def draw(self):
        print('Drawing at position:', self.x, self.y)

In [82]:
class MoveableAdapter(Root):
    def __init__(self, x, y, **kwds):
        self.movable = Moveable(x, y)
        super().__init__(**kwds)
    def draw(self):
        self.movable.draw()
        super().draw()

class MovableColoredShape(ColoredShape, MoveableAdapter):
    pass

MovableColoredShape(color='red', shapename='triangle',
                    x=10, y=20).draw()

Drawing.  Setting color to: red
Drawing.  Setting shape to: triangle
Drawing at position: 10 20


# **super** means: **the next one in the line** (from linearization)
## two constraints: children precede their parents and the order of appearance in __bases__ is respected.

In [69]:
class A:
    def save(self): pass

class B(A): pass

class C(A):
    def save(self): pass

class D(B, C): pass

In [70]:
help(D)

Help on class D in module __main__:

class D(B, C)
 |  Method resolution order:
 |      D
 |      B
 |      C
 |      A
 |      builtins.object
 |  
 |  Methods inherited from C:
 |  
 |  save(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from A:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

